<a href="https://colab.research.google.com/github/VictorMadu/Stackover-question-tagging/blob/master/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import keras
import sqlite3
import io
import dill
from keras.layers import BatchNormalization, Dropout, Dense, Input, concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [0]:
batch_size = 50000
vectorizer = dill.load(open('/content/drive/My Drive/tag_vectorizer.p', 'rb'))

In [0]:

class TrainGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  def __init__(self, path, val_indices=[]):
    'Initialization'
    sqlite3.register_adapter(np.ndarray, self.adapt_array)
    sqlite3.register_converter("array", self.convert_array)
    self.path = path
    self.conn = sqlite3.connect(self.path, detect_types=sqlite3.PARSE_DECLTYPES, check_same_thread=False)
    self.cur = self.conn.cursor()
    self.shuffle_index = np.arange(self.__len__())
    self.shuffle_index = np.delete(self.shuffle_index, val_indices)
    np.random.shuffle(self.shuffle_index)
    self.number_of_batches = int(np.ceil(self.__len__() // batch_size))
    

  def __len__(self):
    sql_query = """SELECT COUNT(*) FROM data"""
    self.cur.execute(sql_query)
    value = self.cur.fetchone()
    return value[0]


  def __getitem__(self, batch_size):
    'Gets batches of data'
    while True:
      self.counter = 0
      for i in range(self.number_of_batches):
        if self.number_of_batches > self.counter:
          index_batch = self.shuffle_index[batch_size*self.counter:batch_size*(self.counter+1)]
          self.counter += 1
          yield self.__data_generation(index_batch, batch_size)
          print('Batch', i+1, "loaded")
        else: 
          index_batch = self.shuffle_index[batch_size*self.counter:]
          yield self.__data_generation(index_batch, batch_size)
          self.counter = 0
          print('Batch', i+1, "loaded")
          



  def __data_generation(self, index_batch, batch_size):
    'Generates data containing batch_size samples'
    sql_query = """SELECT title_vec, question_vec, code_vec, len_title, len_question, tags FROM data WHERE Id in {}""".format(tuple(index_batch))
    self.cur.execute(sql_query)
    x1 = np.empty((batch_size, 128))
    x2 = np.empty((batch_size, 128))
    x3 = np.empty((batch_size, 128))
    x4 = np.empty((batch_size, 1))
    x5 = np.empty((batch_size, 1))
    y = np.empty((batch_size, 5500))

    for i in range(batch_size):
      line = self.cur.fetchone()
      x1[i, :] = line[0]
      x2[i, :] = line[1]
      x3[i, :] = line[2]
      x4[i, :] = line[3]
      x5[i, :] = line[4]
      y[i, :] = vectorizer.transform([line[5]]).toarray()
    return ([x1, x2, x3, x4, x5], y)

  def adapt_array(arr):
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

  def convert_array(self, text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

In [0]:
class ValidationGenerator(keras.utils.Sequence):
  def __init__(self, path):
    self.path = path
    sqlite3.register_adapter(np.ndarray, self.adapt_array)
    sqlite3.register_converter("array", self.convert_array)
    self.conn = sqlite3.connect(self.path, detect_types=sqlite3.PARSE_DECLTYPES)
    self.cur = self.conn.cursor()
    self.shuffle_index = np.arange(self.__len__())
    np.random.shuffle(self.shuffle_index)


  def __getitem__(self, batch_size):
    index_batch = self.shuffle_index[:batch_size]
    return self.__data_generation(index_batch, batch_size)

  def __len__(self):
    sql_query = """SELECT COUNT(*) FROM data"""
    self.cur.execute(sql_query)
    value = self.cur.fetchone()
    return value[0]
    

  def __data_generation(self, index_batch, batch_size):
    'Generates data containing batch_size samples'
    sql_query = """SELECT title_vec, question_vec, code_vec, len_title, len_question,  tags FROM data WHERE Id in {}""".format(tuple(index_batch))
    self.cur.execute(sql_query)
    x1 = np.empty((batch_size, 128))
    x2 = np.empty((batch_size, 128))
    x3 = np.empty((batch_size, 128))
    x4 = np.empty((batch_size, 1))
    x5 = np.empty((batch_size, 1))
    y = np.empty((batch_size, 5500))

    for i in range(batch_size):
      line = self.cur.fetchone()
      x1[i, :] = line[0]
      x2[i, :] = line[1]
      x3[i, :] = line[2]
      x4[i, :] = line[3]
      x5[i, :] = line[4]
      y[i, :] = vectorizer.transform([line[5]]).toarray()
    return ([x1, x2, x3, x4, x5], y, index_batch)


  def adapt_array(arr):
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

  def convert_array(self, text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

In [0]:
validation = ValidationGenerator("/content/drive/My Drive/Question.sqlite")
X_val, y_val, val_indices = validation.__getitem__(100000)

In [0]:
train = TrainGenerator("/content/drive/My Drive/Question.sqlite", val_indices)

In [0]:
def nn_model():
  print('Building model...')
  seq1 = Input(shape=(128,))
  seq2 = Input(shape=(128,))
  seq3 = Input(shape=(128,))
  seq4 = Input(shape=(1,))
  seq5 = Input(shape=(1,))

  dense1 = Dropout(0.2)(seq1)
  dense1 = Dense(1024, activation='relu')(dense1)
  dense1 = Dense(1024, activation='relu')(dense1)

  dense2 = Dropout(0.2)(seq2)
  dense2 = Dense(1024, activation='relu')(dense2)
  dense2 = Dense(1024, activation='relu')(dense2)

  dense3 = Dropout(0.2)(seq3)
  dense3 = Dense(1024, activation='relu')(dense3)
  dense3 = Dense(1024, activation='relu')(dense3)

  merge = concatenate([dense1, dense2, dense3, seq4, seq5])
  merge = Dropout(0.2)(merge)
  merge = BatchNormalization()(merge)
  merge = Dense(3072, activation='relu')(merge)
  merge = Dense(3072, activation='relu')(merge)

  merge = Dropout(0.2)(merge)
  merge = BatchNormalization()(merge)
  pred = Dense(5500, activation='softmax')(merge)
  
  model = Model(inputs=[seq1, seq2, seq3, seq4, seq5], outputs=pred)

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
es = EarlyStopping(monitor='val_loss', patience=4)
mc = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)


model = nn_model()
history = model.fit_generator(generator=train.__getitem__(batch_size), 
                                  steps_per_epoch=2, 
                                        callbacks = [es, mc], 
                                           validation_data=(X_val, y_val) 
                                           )

Building model...
Epoch 1/1


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


Batch 1 loaded
Batch 2 loaded
Batch 3 loaded
Batch 4 loaded
